In [2]:
## imports
import pandas as pd
import time

In [6]:
# Define the seasons
seasons = [f"{year}-{year+1}" for year in range(1996, 2024)]

# Base URL format for the Champions League stats
real_madrid_base_url = 'https://fbref.com/en/squads/53a2f082/{}/c8/Real-Madrid-Stats-Champions-League'

# Initialize an empty DataFrame to hold all the data
realMadrid_df = pd.DataFrame()
dortmund_df = pd.DataFrame()


In [8]:
# Loop through each season and load the data
for season in seasons:
    url = real_madrid_base_url.format(season)
    try:
        # Read the data from the URL
        season_df = pd.read_html(url, attrs={"id": "matchlogs_for"})[0]
        
        # Add a column for the season
        season_df['Season'] = season
        
        # Concatenate the season's data to the main DataFrame
        realMadrid_df = pd.concat([realMadrid_df, season_df], ignore_index=True)
        
        print(f"Successfully loaded data for season {season}")
    except Exception as e:
        print(f"Failed to load data for season {season}: {e}")
    
    # Sleep for 5 seconds to avoid triggering a "too many requests" error
    time.sleep(5)

Successfully loaded data for season 1996-1997
Successfully loaded data for season 1997-1998
Successfully loaded data for season 1998-1999
Successfully loaded data for season 1999-2000
Successfully loaded data for season 2000-2001
Successfully loaded data for season 2001-2002
Successfully loaded data for season 2002-2003
Successfully loaded data for season 2003-2004
Successfully loaded data for season 2004-2005
Successfully loaded data for season 2005-2006
Successfully loaded data for season 2006-2007
Successfully loaded data for season 2007-2008
Successfully loaded data for season 2008-2009
Successfully loaded data for season 2009-2010
Successfully loaded data for season 2010-2011
Successfully loaded data for season 2011-2012
Successfully loaded data for season 2012-2013
Successfully loaded data for season 2013-2014
Successfully loaded data for season 2014-2015
Successfully loaded data for season 2015-2016
Successfully loaded data for season 2016-2017
Successfully loaded data for seaso

In [22]:
realMadrid_df = realMadrid_df[realMadrid_df['Comp'].isna()]
realMadrid_df = realMadrid_df.dropna(subset=['Result'])
print(realMadrid_df.head())

# Save the combined DataFrame to a CSV file
realMadrid_df.to_csv('real_madrid_champions_league_1996_2024.csv', index=False)

          Date Time Comp        Round  Day Venue Result   GF   GA  \
42  1997-09-17  NaN  NaN  Group stage  Wed  Home      W  4.0  1.0   
43  1997-10-01  NaN  NaN  Group stage  Wed  Away      W  2.0  0.0   
44  1997-10-22  NaN  NaN  Group stage  Wed  Home      W  5.0  1.0   
45  1997-11-05  NaN  NaN  Group stage  Wed  Away      D  0.0  0.0   
46  1997-11-27  NaN  NaN  Group stage  Thu  Away      L  0.0  2.0   

         Opponent  Poss  Attendance Captain Formation            Referee  \
42   no Rosenborg   NaN         NaN     NaN       NaN   Gilles Veissière   
43       pt Porto   NaN         NaN     NaN       NaN    Piero Ceccarini   
44  gr Olympiacos   NaN         NaN     NaN       NaN  Pierluigi Collina   
45  gr Olympiacos   NaN         NaN     NaN       NaN       Hellmut Krug   
46   no Rosenborg   NaN         NaN     NaN       NaN        Paul Durkin   

    Match Report Notes     Season  xG  xGA  
42  Match Report   NaN  1997-1998 NaN  NaN  
43  Match Report   NaN  1997-1998 NaN 